In [ ]:
!pip install -q langchain langchainhub langchain-huggingface langchain-community langchain-core langchain-text-splitters bitsandbytes docx2txt langchain-chroma

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name = 'BAAI/bge-m3'
)

In [ ]:
from langchain_chroma import Chroma

collection_name = 'chroma_tax'

database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
    collection_name=collection_name,
    persist_directory='./chroma_huggingface2'
)

In [ ]:
# LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct

from transformers import BitsAndBytesConfig

# 양자화 -> 모델의 정확도를 크게 해치지 않으면서 크기를 줄이는 것
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True
)

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

chat_model = HuggingFacePipeline.from_model_id(
    model_id = 'LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct',
    task='text-generation',
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        do_sample=False,
        repetition_penalty=1.03
    ),
    model_kwargs={'quantization_config': quantization_config}
)

In [ ]:
llm = ChatHuggingFace(llm=chat_model)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
retriever = database.as_retriever(search_kwargs={"k": 2})

In [ ]:
query = '연봉 5천만원인 거주자의 종합소득에 대한 소득세?'
retriever.invoke(query)

In [ ]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
ai_message = retrieval_chain.invoke({"input": query})

In [ ]:
ai_message